# Module 3: Edit Distance, Alignment & Sequence Assembly
**Programming Homework – Genomic Data Science Specialization (Johns Hopkins)**  
**Course:** Algorithms for DNA Sequencing  
**Specialization:** Bioinformatics – Genomic Data Science  
**Author:** Julian Borges (personal documentation) 
**Module:** 3 – Edit Distance, Alignment, and Overlap Graphs


In [ ]:
import numpy as np
from collections import defaultdict


In [ ]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line.startswith('>'):
                genome += line.strip()
    return genome


In [ ]:
def editDistance(x, y):
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    for i in range(len(x)+1):
        D[i][0] = i
    for j in range(len(y)+1):
        D[0][j] = j
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    return D[-1][-1]


In [ ]:
def approximateMatch(p, t):
    D = []
    for i in range(len(p)+1):
        D.append([0]*(len(t)+1))
    for i in range(len(p)+1):
        D[i][0] = i
    for j in range(len(t)+1):
        D[0][j] = 0
    for i in range(1, len(p)+1):
        for j in range(1, len(t)+1):
            del_cost = D[i-1][j] + 1
            ins_cost = D[i][j-1] + 1
            sub_cost = D[i-1][j-1] + (0 if p[i-1] == t[j-1] else 1)
            D[i][j] = min(del_cost, ins_cost, sub_cost)
    return min(D[len(p)])


In [ ]:
def overlap(a, b, min_length=3):
    start = 0
    while True:
        start = a.find(b[:min_length], start)
        if start == -1:
            return 0
        if b.startswith(a[start:]):
            return len(a) - start
        start += 1


In [ ]:
def readFastq(filename):
    sequences = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name
            seq = fh.readline().rstrip()
            fh.readline()  # skip plus
            fh.readline()  # skip quality
            if len(seq) == 0:
                break
            sequences.append(seq)
    return sequences


In [ ]:
def build_kmer_dict(reads, k):
    index = defaultdict(set)
    for read in reads:
        for i in range(len(read) - k + 1):
            index[read[i:i+k]].add(read)
    return index

def count_overlaps(reads, k):
    index = build_kmer_dict(reads, k)
    overlaps = set()
    for read in reads:
        suffix = read[-k:]
        for candidate in index[suffix]:
            if read != candidate:
                olen = overlap(read, candidate, min_length=k)
                if olen > 0:
                    overlaps.add((read, candidate))
    return overlaps

reads = readFastq('/mnt/data/ERR266411_1.for_asm.fastq')
overlaps = count_overlaps(reads, 30)
print("Number of edges in overlap graph:", len(overlaps))


In [ ]:
outgoing = set()
for a, b in overlaps:
    outgoing.add(a)
print("Number of nodes with at least one outgoing edge:", len(outgoing))
